## For stars dimmer

```sql
with x as
(
	SELECT G.source_id, G.ra, G.dec, G.phot_g_mean_mag, G.bp_rp, G.astrometric_params_solved, G.ruwe, G.phot_g_mean_mag - (0.01426 * POWER(G.bp_rp, 3) - 0.2156 * POWER(G.bp_rp, 2) + 0.01424 * POWER(G.bp_rp, 1) - 0.02704) as v_mag 
    FROM gaiadr3.gaia_source AS G
	WHERE G.phot_g_mean_mag IS NOT NULL
)
SELECT * 
FROM x
WHERE (x.bp_rp IS NOT NULL AND x.v_mag > 16.75 AND x.v_mag <= 18.0) OR (x.bp_rp IS NULL AND x.phot_g_mean_mag > 16.75  AND x.phot_g_mean_mag <= 18.0)
```

correspond to file named `1733607897473O-result.fits`


In [ ]:
import pathlib
import struct
import numpy as np
import pandas as pd
from astropy.table import Table
from py.geodesic import radec2xyz, GeodesicGrid
import tqdm
from py.gaia import gbprp_to_bv
import trimesh
from astropy.time import Time


gaia_base_path = pathlib.Path("./gaia_query_results")
starcatalog_base_path = pathlib.Path("./star_catalogs")

# =============================================================================
# Process the Gaia DR3 catalog
# =============================================================================
gaia_t = Table.read(gaia_base_path / "1733607897473O-result.fits", format="fits")
good_astrometry_idx = (gaia_t["astrometric_params_solved"] == 31) | (
    gaia_t["astrometric_params_solved"] == 95
) & (gaia_t["ruwe"] < 1.4)
df_gaia = gaia_t.to_pandas()
b_v = np.zeros(len(df_gaia)) * np.nan

# read synthetic photometry
synth_phot_df = pd.read_csv(gaia_base_path / "Gaia_XP_JKC.csv")
# try to get synthetic photometry
matched_source_id, idx1, idx2 = np.intersect1d(
    df_gaia["source_id"].values, synth_phot_df["source_id"].values, return_indices=True
)
# if we have synthetic photometry, use it
b_v[idx1] = (synth_phot_df.loc[idx2, "Jkc_mag_B"].values - synth_phot_df.loc[idx2, "Jkc_mag_V"].values)
# baseline B-V calculation, inferior to the synthetic photometry
_, b_v_est = gbprp_to_bv(df_gaia["phot_g_mean_mag"], df_gaia["bp_rp"], red_correction=True)
b_v = np.where(np.isnan(b_v), b_v_est, b_v)
df_gaia.fillna({"b_v": b_v_est}, inplace=True)
df_gaia["b_v"] = b_v

# make df_gaia
df = pd.DataFrame(
    index=range(len(df_gaia)),
    data={
        "source_id": df_gaia["source_id"].values,
        "ra": df_gaia["ra"].values,
        "dec": df_gaia["dec"].values,
        "epoch": np.zeros(len(df_gaia), dtype=float)
        + 2016.0,  # we have propagated to J2000.0
        "b_v": df_gaia["b_v"].values,
        "vmag": df_gaia["v_mag"].values,
    },
)

# fill bad data with 0
df = df.fillna(0)
assert df["source_id"].dtype == np.int64

min_vmag = [16.75]
max_vmag = [18.0]
major_version = [0]
minor_version = [3]
data_type = [2]
# mag_ranges = [12800]
# mag_steps = [256]
levels = [8]
suffixes = [None, None]
catalog_epoch_jd_fromJ2000 = Time(2016, format="jyear").jd
# =============================================================================
# Write to file
# =============================================================================
for lv, datatype, majver, minver, min_v, max_v, suff in zip(
    levels,
    data_type,
    major_version,
    minor_version,
    min_vmag,
    max_vmag,
    suffixes,
):
    if suff is not None:
        suff = f"_{suff}"
    else:
        suff = ""
    df6 = df[(df["vmag"] > min_v) & (df["vmag"] <= max_v)].reset_index(drop=True)
    print(len(df6))

    # =============================================================================
    # Deal with geodesic grid
    # =============================================================================
    grid = GeodesicGrid(level=lv)

    # setup ray-tracing
    xyz = radec2xyz(df6["ra"], df6["dec"]).T
    vertices = np.vstack(grid.vertices[-1])
    faces = np.vstack(grid.faces[-1])
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    ray_origins = np.tile([[0, 0, 0]], (len(df6),1))
    ray_directions = np.array(xyz)
    intersected_faces, ray_indices = mesh.ray.intersects_id(
        ray_origins=ray_origins,
        ray_directions=ray_directions,
        multiple_hits=False,
    )
    df6.loc[ray_indices, f"zone{lv}"] = intersected_faces
    # find which number ray_indices are missing
    missing = np.setdiff1d(np.arange(len(df6)), ray_indices)
    # if there are missing values, we need to do a more expensive search with algebra
    zones = grid.search_zone(xyz[missing])
    df6.loc[missing, f"zone{lv}"] = zones

    # sort by zone, within each zone sort by vmag
    df6 = df6.sort_values([f"zone{lv}", "vmag"]).reset_index(drop=True)

    f = open(
        starcatalog_base_path / f"./stars_{lv}_{datatype}v{majver}_{minver}{suff}.cat", "w+b"
    )

    f.write(b"\n\x04_\x83")  # Magic Number
    f.write(np.int32(datatype).tobytes())  # Data Type
    f.write(np.int32(majver).tobytes())  # Major Version
    f.write(np.int32(minver).tobytes())  # Minor Version
    f.write(np.int32(lv).tobytes())  # Level
    f.write(np.int32(min_v * 1000).tobytes())  # Magnitude Minimum
    f.write(np.float32(catalog_epoch_jd_fromJ2000).tobytes())  # Catalog Epoch
    n_zones = 20 * 4**lv + 1  # plus 1 global zone

    # count number of stars in each zone in df6
    zone_info = df6[f"zone{lv}"].value_counts().sort_index()
    for z in range(n_zones):
        f.write(struct.pack("i", zone_info.get(z, 0)))

    max_records = sum(zone_info)

    if datatype == 2:
        # round to nearest integer
        df6["ra_36000"] = (df6["ra"] * 36_000).round().astype(int)
        df6["dec_36000"] = ((df6["dec"] + 90) * 36_000).round().astype(int)
        df6["b_v_truncated"] = ((df6["b_v"] + 1) * 40).round().astype(int)  # designed for -1-5 mag with stepping of 0.025 mag
        # crop the range to -1 to 5, this range is big enough in my opnion
        df6["b_v_truncated"] = df6["b_v_truncated"].clip(0, 240)
        df6["vmag_truncated"] = ((df6["vmag"] - 16.) * 50).round().astype(int)  # designed for 16-21 mag with stepping of 0.02 mag
        for i in tqdm.tqdm(range(max_records)):
            # star_header = (
            #     "source_id",
            #     "x0",
            #     "x1",
            #     "b_v",
            #     "mag",
            # )
            f.write(struct.pack("q", df6.loc[i, "source_id"]))

            f.write(
                struct.pack("<I", df6.loc[i, "ra_36000"])[0:3]
                + struct.pack("<I", df6.loc[i, "dec_36000"])[0:3]
            )
            f.write(
                struct.pack(
                    "BB",
                    df6.loc[i, "b_v_truncated"],
                    df6.loc[i, "vmag_truncated"],
                )
            )
    else:
        raise ValueError("Data Type must be 2")
    f.close()

122908184


100%|██████████| 122908184/122908184 [57:29<00:00, 35626.70it/s] 
